In [1]:
import pandas as pd
import sys,os
from tqdm import tqdm
sys.path.insert(0, '/home/shenwanxiang/Research/aliyun_sync/COMPASS/')
from baseline.immnue_score.scorer import ssGSEA

from compass.utils import plot_embed_with_label
from compass import PreTrainer, FineTuner, loadcompass #, get_minmal_epoch
from compass.utils import plot_embed_with_label,plot_performance, score2
from compass.tokenizer import CANCER_CODE, CONCEPT

/home/shenwanxiang/anaconda3/envs/compass/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = '/home/shenwanxiang/Research/aliyun_sync/COMPASS/paper/00_data/'
df_label = pd.read_pickle(os.path.join(data_path, 'ITRP.PATIENT.TABLE.ALIGN'))
df_tpm = pd.read_pickle(os.path.join(data_path, 'ITRP.TPM.TABLE'))
df_tpm.shape, df_label.shape

dfcx = df_label.cancer_type.map(CANCER_CODE).to_frame('cancer_code').join(df_tpm)

In [8]:
df_label.to_csv('./data/ITRP_clinical.csv')

In [ ]:
gmt29  = pd.read_excel('./Fges_29_gmt.xlsx',sheet_name= 'Gene Description',skiprows=1)
gmt29 = gmt29.groupby(['Gene signature']).Gene.apply(lambda x:x.tolist())

res=[]
for name, g_list in tqdm(gmt29.items()):
    gsea = ssGSEA(g_list, name)
    score = gsea.fit_transform(df_tpm)
    res.append(score) 

dfc = pd.concat(res,axis=1)
dfc.columns = dfc.columns.map(lambda x:x.replace('NES_', ''))
dfc.loc[df_label.index].to_csv('./data/ITRP_ssGSEA_Fegs29.csv')

In [41]:
def _connect(x):
    from itertools import chain
    return list(set(chain(*[i.split(':') for i in x])))
    
concept_genes = CONCEPT.groupby('BroadCelltypePathway').Genes.apply(lambda x:x.tolist()).apply(_connect)

res=[]
for name, g_list in tqdm(concept_genes.items()):
    gsea = ssGSEA(g_list, name)
    score = gsea.fit_transform(df_tpm)
    res.append(score) 

dfc = pd.concat(res,axis=1)
dfc.columns = dfc.columns.map(lambda x:x.replace('NES_', ''))
dfc = dfc[CONCEPT.groupby('BroadCelltypePathway').Concept_index.mean().sort_values().index]
dfc.loc[df_label.index].to_csv('./data/ITRP_ssGSEA_concept43.csv')

43it [06:22,  8.88s/it]


In [52]:
pretrainer = loadcompass('../../../checkpoint/latest/pretrainer.pt', map_location = 'cpu')
dfg, dfc = pretrainer.extract(dfcx,  batch_size= 128)

100%|#####################################################################################| 9/9 [00:28<00:00,  3.21s/it]


In [54]:
dfc.to_csv('./data/ITRP_COMPASS_concept44.csv')

In [13]:
# leave-cohort-out feature extraction
## to extract the concept features from the model that was not finetuned on this cohort
dfl = df_label
c = []
for cohort in dfl.cohort.unique():
    model_path = 'https://www.immuno-compass.com/download/model/LOCO/pft_leave_%s.pt' % cohort
    finetuner = loadcompass(model_path, map_location='cpu')
    print(cohort)
    dfcx1 = dfcx.loc[dfl[dfl.cohort == cohort].index]
    dfg1, dfc1 = finetuner.extract(dfcx1,  batch_size= 128)
    c.append(dfc1)

Downloading...
From: https://www.immuno-compass.com/download/model/LOCO/pft_leave_IMVigor210.pt
To: /tmp/tmpk5eztk07


100%|███████████████████████████████████████████████████████████████████████████████| 35.1M/35.1M [00:51<00:00, 679kB/s]


Model downloaded to: /tmp/tmpk5eztk07
IMVigor210


100%|#####################################################################################| 3/3 [00:07<00:00,  2.53s/it]
Downloading...
From: https://www.immuno-compass.com/download/model/LOCO/pft_leave_Rose.pt
To: /tmp/tmpb8d4v2bx


100%|███████████████████████████████████████████████████████████████████████████████| 35.1M/35.1M [00:56<00:00, 621kB/s]


...

Model downloaded to: /tmp/tmp4ynnjemq
MGH


100%|#####################################################################################| 1/1 [00:01<00:00,  1.03s/it]
Downloading...
From: https://www.immuno-compass.com/download/model/LOCO/pft_leave_IMmotion150.pt
To: /tmp/tmp9xtd1dpk


100%|███████████████████████████████████████████████████████████████████████████████| 35.1M/35.1M [00:52<00:00, 666kB/s]


Model downloaded to: /tmp/tmp9xtd1dpk
IMmotion150


100%|#####################################################################################| 2/2 [00:04<00:00,  2.15s/it]


In [15]:
dfc = pd.concat(c).loc[dfcx.index]
dfc.to_csv('./data/ITRP_PFT_COMPASS_concept44.csv')

,CANCER,Bcell_general,Memory_Bcell,Naive_Bcell,Plasma_cell,CD4_Tcell,CD8_Tcell,Memory_Tcell,Naive_Tcell,Tcell_general,...,Pancreatic,Pneumocyte,Apoptosis_pathway,IFNg_pathway,TGFb_pathway,Cytokine,Cell_proliferation,TLS,Genome_integrity,Reference
Index,,,,,,,,,,,,,,,,,,,,,
IMVigor210-0257bb-ar-0257bbb,1.456546,0.019397,1.381796,-0.370585,0.452068,0.678281,0.732779,0.911525,1.192506,0.571735,...,0.790673,0.379837,1.044820,0.542234,0.439366,0.530098,0.551015,0.533078,0.883911,1.105827
IMVigor210-025b45-ar-025b45c,1.456528,0.214744,1.306711,-0.387404,0.565886,0.416493,0.705398,1.013120,1.334937,0.523259,...,0.606107,0.403372,1.020884,0.440796,0.726907,0.583664,0.371870,0.622946,0.755367,1.102651
IMVigor210-032c64-ar-032c642,1.456583,0.022168,1.483314,-0.357910,0.198185,0.697679,0.702471,1.030406,1.296986,0.806432,...,0.931995,0.377759,1.073285,0.862106,0.587562,0.443517,0.615876,0.571930,0.954822,1.109750
IMVigor210-0571f1-ar-0571f17,1.456573,0.221132,1.711198,0.211879,0.351435,0.385675,0.909300,1.076376,1.729098,1.225904,...,0.922287,0.460977,0.912766,0.863578,0.441387,0.522413,0.528561,1.264815,0.821885,1.097564
IMVigor210-065890-ar-0658907,1.456561,0.031006,1.477538,-0.373197,0.113452,0.662746,0.714458,0.863025,1.186389,0.698400,...,1.001964,0.445109,1.130237,0.684552,0.349045,0.530858,0.473466,0.487399,0.860129,1.116720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
IMmotion150-p97816-ar-1270d,-0.329531,0.063988,1.028003,-0.565454,0.128116,0.164539,0.578495,0.486751,0.660887,0.595266,...,0.469923,0.432903,0.548849,0.506794,0.483246,0.396237,0.267659,0.470257,0.585417,0.939150
IMmotion150-p97909-ar-c7295,-0.329445,0.164098,0.822569,-0.597204,0.626622,0.246451,0.479684,0.615778,0.347046,0.304133,...,0.376021,0.439916,0.562050,0.232171,0.408161,0.393197,0.218544,0.244641,0.529493,0.946691
IMmotion150-p98637-ar-bcb5a,-0.329409,0.075307,0.958596,-0.519167,0.467963,0.184852,0.602223,0.615597,1.018818,0.850902,...,0.473836,0.378169,0.524828,0.759137,0.385891,0.429615,0.201687,0.430165,0.555165,0.935734
